In [ ]:
import h5py
import numpy as np
from sklearn.model_selection import KFold
fh5 = h5py.File('E:/data/fmap2m.h5','r')
img = fh5['map']
img = np.array(img)
lab = fh5['lab']
lab = np.array(lab)

In [ ]:
img = img.reshape(1143, 32, 32, 1)

In [ ]:
lab = lab[0:1143]

In [ ]:
from keras.utils import to_categorical
train_labels = to_categorical(lab)
train_labels.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(img,lab,test_size=0.2,random_state=1)

In [ ]:
from keras import models
from keras import layers
from keras.models import Sequential,Input
from keras import optimizers
import keras
from keras.layers import Conv2D,MaxPooling2D,Dropout,BatchNormalization,Flatten,Dense,Lambda,Multiply,AveragePooling2D,Activation
from keras.models import Model

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=10),
    keras.callbacks.ModelCheckpoint(filepath='e:/data/rmap.h5',monitor='val_loss',save_best_only=True)   ##保存最佳模型
    ]

In [ ]:
def Conv2d_BN(x, nb_filter,kernel_size, padding='valid',strides=(1,1),data_format='channels_last',dilation_rate=(1,1),
              activation='relu',name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,data_format='channels_last',activation='relu',
               dilation_rate=dilation_rate,name=conv_name)(x)
    x = BatchNormalization(name=bn_name)(x)
    
    return x

In [ ]:
def InceptionB(x,nb_filter):
    branch_1x1_t = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_1x1_s = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_1x1_s = Activation('sigmoid')(branch_1x1_s)
    branch_1x1_c = keras.layers.Multiply()([branch_1x1_t,branch_1x1_s])
    
    branch_3x3_t = Conv2d_BN(x,nb_filter,(3,3), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_3x3_s = Conv2d_BN(x,nb_filter,(3,3), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_3x3_s = Activation('sigmoid')(branch_3x3_s)
    branch_3x3_c = keras.layers.Multiply()([branch_3x3_t,branch_3x3_s])
       
    branch_5x5_t = Conv2d_BN(x,nb_filter,(5,5), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_5x5_s = Conv2d_BN(x,nb_filter,(5,5), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_5x5_s = Activation('sigmoid')(branch_5x5_s)
    branch_5x5_c = keras.layers.Multiply()([branch_5x5_t,branch_5x5_s])
    

    x = keras.layers.Concatenate(axis=3)([branch_1x1_c,branch_3x3_c,branch_5x5_c]) #

    return x

In [ ]:
def InceptionC(x,nb_filter):
    branch_1x1_t = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_1x1_s = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_1x1_s = Activation('sigmoid')(branch_1x1_s)
    branch_1x1_c = keras.layers.Multiply()([branch_1x1_t,branch_1x1_s])
    
    branch_3x3_t = Conv2d_BN(x,nb_filter,(3,3), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_3x3_s = Conv2d_BN(x,nb_filter,(3,3), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_3x3_s = Activation('sigmoid')(branch_3x3_s)
    branch_3x3_c = keras.layers.Multiply()([branch_3x3_t,branch_3x3_s])
    

    x = keras.layers.Concatenate(axis=3)([branch_1x1_c,branch_3x3_c]) #branch_1x1_c,

    return x

In [ ]:
def TestNet(input_shape=None):
    inpt = Input(shape=input_shape)

    x = Conv2d_BN(inpt,32,(3,3),strides=(1,1),padding='same')
    x = InceptionB(x,32)
    x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
    x = InceptionB(x,32)
    x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
    x = InceptionC(x,64)
    x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
    x = InceptionC(x,64)
    x = AveragePooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
    x = Dropout(0.3)(x)
    x = Flatten()(x)
    x = Dense(128,activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(1,activation='sigmoid')(x)
    model = Model(inpt,x,name='inception')
    
 
    return model

In [ ]:
model = TestNet(input_shape=(32,32,1))
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model    
plot_model(model, 'd:\\h5\\model_3d.png')

In [ ]:
plot_model(model, 'd:\\h5\\model_3d.png',show_shapes=True)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-3),#
              metrics=['acc'] 
             )

In [ ]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
cvscores = []
for train, test in kfold.split(img, lab):
    model = TestNet(input_shape=(32,32,1))
    model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-3),#
              metrics=['acc'])
    model.fit(img[train], lab[train], epochs=30, batch_size=32, verbose=0)
    scores = model.evaluate(img[test], lab[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

In [ ]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))